## Agente LangChain — Clima em Tempo Real

Este notebook implementa um agente que consulta o clima atual de qualquer lugar do mundo usando LLM no OCI + API Open-Meteo.

### O agente decide sozinho:

- Latitude
- Longitude
- Quando chamar a API de clima

### Funcionalidades

- Consulta clima em tempo real (Open-Meteo)
- Agente com autonomia de coordenadas
- Processamento com LLM via OCI
- Busca inteligente com LangChain (ReAct)

### Passo 1 - Instalando dependencias

In [6]:
!python -m pip install langchain-core langchain==1.1.0 langchain-openai==1.1.10
!python -m pip install requests


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Passo 2 - Tool de Clima (Open-Meteo)

O agente só precisa fornecer latitude e longitude.

In [42]:
import requests
from langchain_core.tools import tool

@tool
def get_weather(latitude: str, longitude: str) -> str:
    """Obtém o clima atual usando latitude e longitude."""
    
    url = (
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        f"&current=temperature_2m,wind_speed_10m"
    )

    response = requests.get(url)
    data = response.json()
    
    return str(data)


### Passo 3 - Configuração do Agente

In [3]:
import json
from langchain_core.callbacks import BaseCallbackHandler

class CleanAgentCallback(BaseCallbackHandler):
    def on_tool_start(self, serialized, input_str, **kwargs):
        tool_name = serialized.get("name", "unknown_tool")
        print(f"\n TOOL: {tool_name}")
        print(f"   ↳ input: {input_str}")

    def on_tool_end(self, output, **kwargs):
        print("TOOL RESULT (resumo):")

        content = getattr(output, "content", output)

        try:
            data = json.loads(content)

            if "current" in data:
                temp = data["current"].get("temperature_2m")
                wind = data["current"].get("wind_speed_10m")

                print(f"   ↳ Temperatura: {temp}°C")
                print(f"   ↳ Vento: {wind} km/h")

        except Exception:
            text = str(content)
            print("   ↳", text[:200])


### Passo 4 - LLM

In [35]:
from dotenv import load_dotenv
import os

load_dotenv()

base_url = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com/20231130/actions/v1"  # ou sua URL
api_key = os.getenv("OCI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="xai.grok-4-fast-non-reasoning",
    temperature=0.2,
    max_tokens=500,
    streaming=False,
    api_key=api_key,
    base_url=base_url
)


### Passo 5 - Lista de Tools

In [36]:
tools = [
    get_weather
]


### Passo 6 - Prompt ReAct

Aqui instruímos o agente a:

- Escolher latitude/longitude
- Analisar o clima
- Dar insights úteis

In [37]:
react_prompt_template = """You are a helpful weather assistant.

You can choose any latitude and longitude needed to answer.

IMPORTANT:
- Always choose coordinates yourself
- After using the tool, analyze the data
- Do not output raw JSON
- Provide insights

{tools}

Begin!
"""


### Passo 7 - Criação do Agente

In [38]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=react_prompt_template
)


### Passo 8 - Função de Extração da Resposta Final

In [39]:
from langchain_core.messages import AIMessage

def extract_final_output(agent_response):
    final_text = None
    total_tokens = None

    for msg in reversed(agent_response["messages"]):
        if isinstance(msg, AIMessage):

            if msg.content and final_text is None:
                if "Final Answer:" in msg.content:
                    final_text = msg.content.split("Final Answer:",1)[1].strip()
                else:
                    final_text = msg.content.strip()

            if total_tokens is None:
                total_tokens = agent_response["messages"][-1].usage_metadata["total_tokens"]

        if final_text and total_tokens:
            break

    return final_text, total_tokens


### Teste 1 — Clima em Paris

In [43]:
callback = CleanAgentCallback()

pergunta = "Como está o clima agora em Paris?"

response = agent.invoke(
    {"messages": pergunta},
    config={"callbacks": [callback]}
)

final_answer, total_tokens = extract_final_output(response)

print("\nFinal answer:")
print(final_answer)

print("\nMetadados:")
print(f"total tokens: {total_tokens}")



 TOOL: get_weather
   ↳ input: {'latitude': '48.8566', 'longitude': '2.3522'}
TOOL REAL EXECUTADA
TOOL RESULT (resumo):
   ↳ {'latitude': 48.86, 'longitude': 2.3599997, 'generationtime_ms': 0.044345855712890625, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 36.0, 'current_units': {

Final answer:
### Clima Atual em Paris

Olá! Baseado nos dados meteorológicos mais recentes para Paris (coordenadas aproximadas: 48.86°N, 2.36°E), o clima está **parcialmente nublado** com uma temperatura de **14.8°C**. Há uma brisa leve com ventos de cerca de **10.8 km/h**, o que torna o dia agradável para passeios ao ar livre, mas pode haver variações ao longo da tarde.

**Insights rápidos:**
- Essa temperatura é um pouco acima da média para esta época do ano em Paris, sugerindo um dia ameno e primaveril.
- Recomendo verificar atualizações em tempo real, pois o tempo na região pode mudar rapidamente devido à proximidade do Sena e influências urbanas.

Se precisa

#### Resposta do agente "crua" sem tratar a mensagem

In [44]:
print(response)

{'messages': [HumanMessage(content='Como está o clima agora em Paris?', additional_kwargs={}, response_metadata={}, id='8f61f5bc-3676-4fd9-b7bf-f229618dcce4'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 427, 'total_tokens': 468, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 426, 'text_tokens': 427, 'image_tokens': 0}, 'num_sources_used': 0, 'cost_in_usd_ticks': 420000}, 'model_provider': 'openai', 'model_name': 'xai.grok-4-fast-non-reasoning', 'system_fingerprint': 'fp_87723bc410', 'id': 'da261b13-5573-29a4-0c4f-d8c3b02c30b4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c8b80-872f-7b41-9edc-cac82cb9e815-0', tool_calls=[{'name': 'get_weather', 'args': {'latitude': '48.8566', 'longitude': '2.3522'}, 'id': 'call_18592831', '

### Teste 2 — Clima no Rio

In [45]:
pergunta = "Está calor no Rio de Janeiro agora?"

response = agent.invoke(
    {"messages": pergunta},
    config={"callbacks": [callback]}
)

final_answer, total_tokens = extract_final_output(response)

print("\nFinal answer:")
print(final_answer)

print("\nMetadados:")
print(f"total tokens: {total_tokens}")



 TOOL: get_weather
   ↳ input: {'latitude': '-22.9068', 'longitude': '-43.1729'}
TOOL REAL EXECUTADA
TOOL RESULT (resumo):
   ↳ {'latitude': -22.875, 'longitude': -43.25, 'generationtime_ms': 0.09322166442871094, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 12.0, 'current_units': {'t

Final answer:
Sim, está bem quente no Rio de Janeiro agora! A temperatura atual é de aproximadamente 30.6°C, com ventos leves de cerca de 9 km/h. Isso é típico de um dia quente na cidade, então recomendo se hidratar e usar protetor solar se for sair. Se precisar de mais detalhes ou previsão, é só pedir!

Metadados:
total tokens: 699
